In [1]:
import os

os.chdir("..")

In [2]:
from src.datasets import EarthSurfaceWater
from tqdm import tqdm
import torch
import rasterio
import numpy as np
import os

In [3]:
os.makedirs("benchmark-datasets/earth_surface_water/tra-patched_scene", exist_ok=True)
os.makedirs("benchmark-datasets/earth_surface_water/tra-patched_truth", exist_ok=True)
os.makedirs("benchmark-datasets/earth_surface_water/val-patched_scene", exist_ok=True)
os.makedirs("benchmark-datasets/earth_surface_water/val-patched_truth", exist_ok=True)

In [4]:
image_profile = {
    "driver": "GTiff",
    "dtype": "uint16",
    "compress": "lzw",
    "predictor": 2,
    "nodata": 0,
    "width": 256,
    "height": 256,
    "count": 6,
}

mask_profile = {
    "driver": "GTiff",
    "dtype": "uint8",
    "compress": "lzw",
    "predictor": 2,
    "nodata": None,
    "width": 256,
    "height": 256,
    "count": 1,
}


ds = EarthSurfaceWater(
    root="benchmark-datasets/earth_surface_water",
    split="train",
    use_patched_version=False,
    pad_sizes=True,
)

j = 0
for i in tqdm(range(len(ds))):
    image = ds[i]["image"]
    mask = ds[i]["mask"]

    for x in range(0, image.shape[1], 256):
        for y in range(0, image.shape[2], 256):
            image_patch = image[:, x : x + 256, y : y + 256]
            mask_patch = mask[x : x + 256, y : y + 256]

            if torch.all(image_patch == 0):
                continue

            with rasterio.open(
                os.path.join(ds.root, "tra-patched_scene", f"image_patch_{j}.tif"),
                "w",
                **image_profile,
            ) as dst:
                dst.write(image_patch.numpy().astype(np.uint16))

            with rasterio.open(
                os.path.join(ds.root, "tra-patched_truth", f"mask_patch_{j}.tif"),
                "w",
                **mask_profile,
            ) as dst:
                dst.write(mask_patch.numpy().astype(np.uint8), 1)

            j += 1

100%|██████████| 64/64 [00:22<00:00,  2.91it/s]


In [5]:
ds = EarthSurfaceWater(
    root="benchmark-datasets/earth_surface_water",
    split="test",
    use_patched_version=False,
    pad_sizes=True,
)

j = 0
for i in tqdm(range(len(ds))):
    image = ds[i]["image"]
    mask = ds[i]["mask"]

    for x in range(0, image.shape[1], 256):
        for y in range(0, image.shape[2], 256):
            image_patch = image[:, x : x + 256, y : y + 256]
            mask_patch = mask[x : x + 256, y : y + 256]

            if torch.all(image_patch == 0):
                continue

            with rasterio.open(
                os.path.join(ds.root, "val-patched_scene", f"image_patch_{j}.tif"),
                "w",
                **image_profile,
            ) as dst:
                dst.write(image_patch.numpy().astype(np.uint16))

            with rasterio.open(
                os.path.join(ds.root, "val-patched_truth", f"mask_patch_{j}.tif"),
                "w",
                **mask_profile,
            ) as dst:
                dst.write(mask_patch.numpy().astype(np.uint8), 1)

            j += 1

100%|██████████| 31/31 [00:11<00:00,  2.76it/s]
